In [ ]:
import pygame
from collections import deque
from random import random
from math import inf,sqrt

BLACK = (0,0,0)
WHITE = (255,255,255)
RED = (255,0,0)
GREEN = (0,255,0)
BLUE = (0,0,255)
YELLOW = (255,255,0)

screen_width= 600
n_lines = 50  
n_columns = n_lines
percent_wall = 0.24 #chance of one in the wall
W = screen_width/ n_lines
H = screen_width/ n_columns

def pop_smaller_dist(list):  #percorre a list de tras pra frente e retorna o no com a smaller dist     
    index = len(list) - 1
    smaller = list[index]  
    for i in range(index,-1,-1):
        if list[i].dist < smaller.dist:
            smaller = list[i]
            index = i
    return list.pop(index)

def heuristic(a,b,func):
    if func == "dist_euclid":
        return sqrt((a.x - b.x)**2 + (a.y - b.y)**2)
    if func == "dist_manhattan":
        return abs(a.x - b.x) + abs(a.y - b.y)

class Node():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.visited = False
        self.neighbors = []
        self.dist = inf
        self.g = inf
        self.pai = None
        self.wall = False

    def __repr__(self):
        return (f"Node({self.x} {self.y})") #debug

    def show(self,color):
        global screen
        if self.wall == False:
            pygame.draw.rect(screen, color, (self.x * W, self.y * H , W-1, H-1))
        else:
            pygame.draw.rect(screen, BLACK, (self.x * W, self.y * H , W-1, H-1))
        pygame.display.update((self.x * W, self.y * H , W-1, H-1)) #f updates only the rect that has been changed

    def add_neighbors(self,grid): #implement diagonal neighbors dps
        self.neighbors = [] #tem q zero the neighbors for when to shuffle the walls
        if self.wall == False: #wall has no neighbor
            x = self.x
            y = self.y
            if (x > 0 and grid[x-1][y].wall == False): 
                self.neighbors.append(grid[x-1][y])

            if (y > 0 and grid[x][y-1].wall == False): #left
                self.neighbors.append(grid[x][y-1])

            if (x < (n_lines - 1) and grid[x+1][y].wall == False): #low
                self.neighbors.append(grid[x+1][y]) 

            if (y < (n_columns - 1) and grid[x][y+1].wall == False ): #right
                self.neighbors.append(grid[x][y+1]) 

    def mix(self): 
        self.wall = False
        if (random() < percent_wall):
            self.wall = True

class Grid():
    def __init__(self): 
        self.grid = []
        for i in range(n_lines): 
            self.grid.append([])
            for j in range(n_columns):
                self.grid[i].append(Node(i,j))
                self.grid[i][j].show(WHITE)
        
    def init(self): #configure neighbors and show the grid    
        self.start.wall = False
        self.end.wall = False
        for i in range(n_lines):
            for j in range(n_columns):
                self.grid[i][j].add_neighbors(self.grid)
                self.grid[i][j].show(WHITE)
        self.start.show(BLUE)
        self.end.show(BLUE)
        
    def grid_aleatoria(self): #muda os walls de lugar
        for i in range(n_lines):
            for j in range(n_columns):
                self.grid[i][j].mix()
        self.init() #recalculates neighbors and shows the new grid

    def set_start(self,a,b):
        self.start = self.grid[a][b]
        self.start.show(BLUE)
        
    def set_end(self,a,b):
        self.end = self.grid[a][b]
        self.end.show(BLUE)

    def set_wall(self,a,b):
        self.grid[a][b].wall = True
        self.grid[a][b].show(BLACK)
    
    
    def best_first_search(self,func_dist):
        for i in range(n_lines):
            for j in range(n_columns):
                self.grid[i][j].dist = inf
                self.grid[i][j].pai = None
                self.grid[i][j].show(WHITE)
                
        self.start.dist =heuristic(self.start, self.end, func_dist)
        pq = [] #defines an empty priority queue
        pq.append(self.start)
        while pq: # while the queue is not empty
            u = pop_smaller_dist(pq) #selecione u em S, tal que dist[u] é minima
            u.show(GREEN)
            if u == self.end:
                self.rebuild_path()
                return
            for v in u.neighbors: #for each neighbor v of u
                if v.dist == inf: #even if v has not been visited yet
                    v.dist =heuristic(v, self.end, func_dist)
                    pq.append(v)
                    v.pai = u
        print("There is no path")
        self.start.show(BLUE)
        self.end.show(BLUE)
        
    def a_star(self,func_dist): 
        for i in range(n_lines):
            for j in range(n_columns):
                self.grid[i][j].dist = inf
                self.grid[i][j].g = inf
                self.grid[i][j].show(WHITE)
        # f = g + h
        self.start.g = 0
        self.start.dist = self.start.g +heuristic(self.start,self.end,func_dist) 

        openSet = []
        openSet.append(self.start)

        while openSet:
            atual = pop_smaller_dist(openSet) #This operation can occur in O(1) time if openSet is a min-heap or a priority queue
            atual.show(GREEN)

            if atual == self.end:
                self.rebuild_path()
                return
            
            for neighbor in atual.neighbors:
                g_temp = atual.g + 1
                if g_temp < neighbor.g:
                    neighbor.pai = atual
                    neighbor.g = g_temp
                    neighbor.dist =  neighbor.g +heuristic(neighbor,self.end,func_dist)
                    if neighbor not in openSet:
                        openSet.append(neighbor)
                        neighbor.show(RED)
        print("There is no path")
        self.start.show(BLUE)
        self.end.show(BLUE)

    def rebuild_path(self):
        path = self.end
        while path:
            path.show(BLUE)
            path = path.pai

    def config_initial(self):
        aux = 0
        print("\n Choose a starting position")
        while True:
            config = pygame.event.wait()
            if config.type == pygame.MOUSEBUTTONDOWN and aux == 0:
                mx,my = pygame.mouse.get_pos()
                mx = int(mx / W)
                my = int(my / H)
                self.set_start(mx,my)
                print("Choose an Ending position")
                aux+=1
            elif config.type == pygame.MOUSEBUTTONDOWN and aux == 1:
                mx,my = pygame.mouse.get_pos()
                mx = int(mx / W)
                my = int(my / H)
                self.set_end(mx,my)
                print("Draw walls and press (Enter) when Finished:")
                aux+=1
            elif pygame.mouse.get_pressed()[0] and aux == 2:
                mx,my = pygame.mouse.get_pos()
                mx = int(mx / W)
                my = int(my / H)
                self.set_wall(mx,my)
            elif config.type == pygame.KEYDOWN and aux == 2:
                if config.key == pygame.K_RETURN:
                    self.init()
                    print("\nCOMANDOS:")
                    print("(Esc) To Leave")
                    print("(e)   Best First Search")
                    print("(a)   A*")
                    break

    def new_grid(self):
        for i in range(n_lines):
            for j in range(n_columns):
                self.grid[i][j].wall = False
                self.grid[i][j].show(WHITE)
        self.config_initial()


def main():
    global screen
    pygame.init()
    screen = pygame.display.set_mode((screen_width,screen_width))
    pygame.display.set_caption("Path Finding")

    main_grid = Grid()
    main_grid.config_initial()

    run = True
    while run:
        event = pygame.event.wait()
        if event.type == pygame.KEYDOWN:
            if   event.key == pygame.K_RIGHT:   main_grid.grid_aleatoria()
            elif event.key == pygame.K_LEFT:    main_grid.new_grid()
            elif event.key == pygame.K_e:       main_grid.best_first_search("dist_euclid")
            elif event.key == pygame.K_a:       main_grid.a_star("dist_manhattan")
            elif event.key == pygame.K_ESCAPE:  run = False 
        elif event.type == pygame.QUIT: run = False #closse window
    pygame.quit()
    return

main()